In [0]:
import numpy as np

import matplotlib.pyplot as plt

import pandas as pd

from matplotlib import gridspec

from sklearn.preprocessing import scale



import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader

from torch.utils.data import Dataset

from sklearn.utils import shuffle
from scipy.special import softmax

from PIL import Image

from PIL import ImageDraw

from skimage import morphology

import math

In [2]:
!rm TorchExperts.py
!wget https://raw.githubusercontent.com/andriygav/EMprior/master/code/TorchExperts.py

rm: cannot remove 'TorchExperts.py': No such file or directory
--2019-10-21 14:36:31--  https://raw.githubusercontent.com/andriygav/EMprior/master/code/TorchExperts.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13093 (13K) [text/plain]
Saving to: ‘TorchExperts.py’

TorchExperts.py     100%[===================>]  12.79K  --.-KB/s    in 0.01s   

2019-10-21 14:36:31 (1.06 MB/s) - ‘TorchExperts.py’ saved [13093/13093]



In [0]:
from tqdm import tqdm as tqdm

In [0]:
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['lines.markersize'] = 6
plt.rcParams['xtick.labelsize'] = 36
plt.rcParams['ytick.labelsize'] = 36
plt.rcParams['legend.fontsize'] = 36
plt.rcParams['axes.titlesize']=36
plt.rcParams['axes.labelsize']=36
plt.rcParams['figure.figsize'] = (24.0, 12.0)
plt.rcParams['font.size'] = 40

plt.rcParams["font.family"] = 'DejaVu Serif'

plt.rcParams['text.usetex'] = False

plt.rcParams['figure.figsize'] = (8.0, 8.0)

In [0]:
from TorchExperts import EachModelLinear, HyperModelNN, MixtureExpert, RegularizeModel

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

# All mixture of expert model

# Experiment

In [0]:
np.random.seed(0)
N = 100
N_noise=200
n = 3
r1 = 0.5
r2 = 1.5
phi = np.linspace(0, 2*np.pi, N)

x0 = 0
y0 = 0

# x = np.hstack([r1*np.sin(phi) + x0, r2*np.sin(phi) + x0])
x = np.hstack([r1*np.sin(phi) + x0, r2*np.sin(phi) + x0, 4*(np.random.rand(N_noise)-0.5) + x0])
x += 0.05*np.random.randn(x.shape[0])
# y = np.hstack([r1*np.cos(phi) + y0, r2*np.cos(phi) + y0])
y = np.hstack([r1*np.cos(phi) + y0, r2*np.cos(phi) + y0, 4*(np.random.rand(N_noise)-0.5) + y0])
y += 0.05*np.random.randn(y.shape[0])



In [52]:
plt.plot(x, y, '.')

plt.grid()

plt.show()

In [0]:
X = np.vstack([x, y, np.ones_like(x)]).T
Y = x**2+y**2

In [54]:
X_tr = torch.FloatTensor(X)
Y_tr = torch.FloatTensor(Y).view([-1,1])

X_tr.shape, Y_tr.shape

(torch.Size([400, 3]), torch.Size([400, 1]))

In [0]:
K = 2
LIST_A = [torch.Tensor([1, 1, 1]) for _ in range(K)]
LIST_w = [torch.Tensor([[0], [0], [1]]), torch.Tensor([[0], [0], [2]])]


LoM = []
for k in range(K):
#     LoM.append(EachModelLinear(input_dim = n, device = device))
    LoM.append(EachModelLinear(input_dim = n, device = device, A = torch.ones(3)*LIST_A[k], w = torch.ones(3,1)*LIST_w[k], regul = True))
#       LoM.append(EachModelLinear(input_dim = n, device = device, A = torch.ones(3)*LIST_A[k], regul = False))

# K += 1
# LoM.append(RegularizeModel(ListOfModels = LoM))

HypPar = {'beta': 1, 'alpha': 1e-5}

HpMd = HyperModelNN(input_dim = n, hidden_dim = 5, output_dim = K, device = device, epochs=500)

model = MixtureExpert(input_dim = n, K = K, HyperParameters = HypPar, HyperModel = HpMd, ListOfModels = LoM, ListOfRegularizeModel = [RegularizeModel(ListOfModels = LoM)], device = device)
# model = MixtureExpert(input_dim = n, K = K, HyperParameters = HypPar, HyperModel = HpMd, ListOfModels = LoM, ListOfRegularizeModel = [], device = device)





In [176]:
%%time
model.fit(X_tr, Y_tr, 50, progress = tqdm)

100%|██████████| 50/50 [02:52<00:00,  3.43s/it]

CPU times: user 2min 45s, sys: 4.07 s, total: 2min 49s
Wall time: 2min 52s


In [177]:
model.HyperParameters

{'alpha': 1e-05, 'beta': tensor(0.8407)}

In [178]:
print(model.ListOfModels[0].A)

print(model.ListOfModels[1].A)

tensor([0.0021, 0.0007, 0.0001])
tensor([5.6278e-05, 5.1399e-05, 8.7738e-05])


In [179]:
print(model.ListOfModels[0].B)

print(model.ListOfModels[1].B)

tensor([[ 1.9865e-03, -7.3606e-07, -2.6930e-06],
        [-7.3606e-07,  7.2227e-04,  7.6134e-07],
        [-2.6930e-06,  7.6134e-07,  1.3193e-04]])
tensor([[ 5.6260e-05, -2.7258e-08,  5.9674e-08],
        [-2.7258e-08,  5.1398e-05, -5.8412e-08],
        [ 5.9674e-08, -5.8412e-08,  8.7434e-05]])


In [180]:
print(model.ListOfModels[0].w_0)

print(model.ListOfModels[1].w_0)

tensor([[-2.7572e-05],
        [-3.2858e-04],
        [ 4.4545e-01]])
tensor([[-6.9662e-05],
        [-3.1761e-04],
        [ 2.9611e+00]])


In [181]:
Text_size = 24

plt.rcParams['lines.linewidth'] = 1
plt.rcParams['lines.markersize'] = 4
plt.rcParams['xtick.labelsize'] = Text_size
plt.rcParams['ytick.labelsize'] = Text_size
plt.rcParams['legend.fontsize'] = Text_size
plt.rcParams['axes.titlesize']=Text_size
plt.rcParams['axes.labelsize']=Text_size
plt.rcParams['figure.figsize'] = (8.0, 8.0)
plt.rcParams['font.size'] = Text_size
plt.rcParams["legend.labelspacing"] = 0.1
plt.rcParams["legend.handletextpad"] = 0.1
#################################################

idx = np.argmax(model.predict(X_tr)[1], axis = 1)
t = np.linspace(0, 2*np.pi, 100)

colors = ['orange', 'green', 'red', 'yellow', 'blue']

for i in np.unique(idx):
    _ = plt.plot(x[idx == i], y[idx == i], '.', color = colors[i])
    
plt.grid()
plt.show()

In [182]:
TEMP = np.array([m.W.data.numpy().reshape(-1) for m in model.ListOfModels])

TEMP

array([[ 1.2590599e-02, -2.3637034e-03,  4.4545308e-01],
       [-2.1248801e-04, -3.4885030e-04,  2.9611421e+00]], dtype=float32)

In [183]:
CIRCLE = np.zeros_like(TEMP)

for i, param in enumerate(TEMP):
    x0, y0 = 0.5*param[:2]
    r = (param[2] + x0**2 + y0**2)**0.5
    CIRCLE[i] = x0, y0, r
    
CIRCLE

array([[ 6.29529962e-03, -1.18185172e-03,  6.67453468e-01],
       [-1.06244006e-04, -1.74425149e-04,  1.72079694e+00]], dtype=float32)

In [184]:
Text_size = 24

plt.rcParams['lines.linewidth'] = 1
plt.rcParams['lines.markersize'] = 4
plt.rcParams['xtick.labelsize'] = Text_size
plt.rcParams['ytick.labelsize'] = Text_size
plt.rcParams['legend.fontsize'] = Text_size
plt.rcParams['axes.titlesize']=Text_size
plt.rcParams['axes.labelsize']=Text_size
plt.rcParams['figure.figsize'] = (8.0, 8.0)
plt.rcParams['font.size'] = Text_size
plt.rcParams["legend.labelspacing"] = 0.1
plt.rcParams["legend.handletextpad"] = 0.1
#################################################

idx = np.argmax(model.predict(X_tr)[1], axis = 1)
t = np.linspace(0, 2*np.pi, 100)

colors = ['orange', 'green', 'red', 'yellow', 'blue']

for i in range(len(model.ListOfModels)):
    _ = plt.plot(x[idx == i], y[idx == i], '.', color = colors[i], label = 'model '+str(i))
    
    x0, y0, r = CIRCLE[i]
    x_plot = x0 + r*np.sin(t)
    y_plot = y0 + r*np.cos(t)
    
    _ = plt.plot(x_plot, y_plot, '-', color = colors[i])
    
plt.grid()

plt.legend(loc = 'best')

# plt.savefig('results/full_infomative.pdf', )

plt.show()